# McCormick Relaxation Arithmetic for Factorable Functions
A convex relaxation $f^{cv}$ of a function $f$ on the convex domain D is a function that is (i) convex on D and (ii) underestimates $f$ on D. Likewise, a concave relaxation $f^{cc}$ of a function $f$ on the convex domain D is a function that is (i) concave on D and (ii) overestimates $f$ on D. McCormick's technique ([McCormick, 1976](https://link.springer.com/article/10.1007/BF01580665)) provides a means for computing pairs of convex/concave relaxations of a multivariate function on interval domains provided that this function is factorable and that the intrinsic univariate functions in its factored form have known convex/concave envelopes or, at least, relaxations.

# Perform McCormick relaxations of a factorable 

Suppose one wants to compute McCormick relaxation of the real-valued function $f(x,y)=x(\exp(x)-y)^2$ for $(x,y)\in [-2,1]\times[-1,2]$, at the point $(x,y)=(0,1)$. 

We start by importing the PyMC library:

In [1]:
import pymc

Then, we proceed by defining the intervals of both variables $x$ and $y$:

In [2]:
X = pymc.McCormick( pymc.Interval(-2.,1.), 0.)
Y = pymc.McCormick( pymc.Interval(-1.,2.), 1.)
print(X)
print(Y)

[ -2.00000e+00 :  1.00000e+00 ] [  0.00000e+00 :  0.00000e+00 ]
[ -1.00000e+00 :  2.00000e+00 ] [  1.00000e+00 :  1.00000e+00 ]


Essentially, the first line means that X is a variable of type pymc.McCormick, belonging to the interval $[−2,1]$, and whose current value is 0. The same holds for the McCormick variable Y, which belonging to the interval $[−1,2]$ and has a value of 1.

Having defined the variables, McCormick relaxations of $f(x,y)=x(exp(x)−y)^2$ on $[−2,1]×[−1,2]$ at $(0,1)$ are simply computed and accessed as:

In [3]:
Z = X*pymc.pow((pymc.exp(X)-Y),2)
print("f relaxations at (0,1):",Z)

f relaxations at (0,1): [ -2.76512e+01 :  1.38256e+01 ] [ -1.38256e+01 :  8.52245e+00 ]


Here, the first pair of bounds correspond to the supporting interval bounds, as obtained with **pymc.Interval**, and which are valid over the entire domain $[−2,1]×[−1,2]$. The second pair of bounds are the values of the convex and concave relaxations at the selected point $(0,1)$. 

In order to describe the convex and concave relaxations on the entire range $[−2,1]×[−1,2]$, it would be necessary to repeat the computations at different points. The current point can be set/modified by using the method pymc.McCormick.set_value(), for instance at $(−1,0)$

In [4]:
X.set_value(-1.0)
Y.set_value(0.0)
print("X:",X)
print("Y:",Y)
Z = X*(pymc.exp(X)-Y)**2
print("f relaxations at (0,1):",Z)

X: [ -2.00000e+00 :  1.00000e+00 ] [ -1.00000e+00 : -1.00000e+00 ]
Y: [ -1.00000e+00 :  2.00000e+00 ] [  0.00000e+00 :  0.00000e+00 ]
f relaxations at (0,1): [ -2.76512e+01 :  1.38256e+01 ] [ -1.75603e+01 :  8.78014e+00 ]


The values of the McCormick convex and concave relaxations of $f(x,y)$, as well as the lower and upper bounds of the supporting interval bounds, can be retrieved, respectively, as:

In [5]:
print("The convex relaxation is:", Z.CV)
print("The concave relaxation is:", Z.CC)
print("The lower bound is:", Z.LB)
print("The upper bound is:", Z.UB)

The convex relaxation is: -17.560280068546856
The concave relaxation is: 8.780140034273428
The lower bound is: -27.651239511697487
The upper bound is: 13.825619755848743


Computing a subgradient of a McCormick relaxation requires specification of the independent variables via the .set_subgradient method, prior to evaluating the function in pymc.McCormick type. Continuing the previous example, the function has two independent variables $x$ and $y$. Defining $x$ and $y$ as the subgradient components $0$ and $1$ (index starts at 0 by convention!), respectively, is done as follows:

In [6]:
X.set_subgradient(2, 0)
Y.set_subgradient(2, 1)

[ -1.00000e+00 :  2.00000e+00 ] [  0.00000e+00 :  0.00000e+00 ] [ ( 0.00000e+00, 1.00000e+00) : ( 0.00000e+00, 1.00000e+00) ]

Similar to above, the McCormick convex and concave relaxations of $f(x,y)$ at $(-1,0)$ along with subgradients of these relaxations are computed as:

In [7]:
Z = X*pymc.pow((pymc.exp(X)-Y),2)
print("f relaxations at (-1,0):",Z)

f relaxations at (-1,0): [ -2.76512e+01 :  1.38256e+01 ] [ -1.75603e+01 :  8.78014e+00 ] [ (-3.19186e+00, 3.70723e+00) : ( 1.59593e+00,-1.85362e+00) ]


The additional information displayed corresponds to, respectively, a subgradient of the McCormick convex underestimator at (-1,0) and a subgradient of the McCormick concave overestimator at (-1,0). In turn, these subgradients can be used to construct affine relaxations on the current range, or passed to a bundle solver to locate the actual minimum or maximum of the McCormick relaxations. 


The subgradients of the McCormick relaxations of $f(x,y)$ at the current point can be retrieved as follows:

In [8]:
print(Z.get_CV_sub())
print(Z.get_CC_sub())
print(Z.get_CV_sub(0))
print(Z.get_CV_sub(1))
print(Z.get_CC_sub(0))
print(Z.get_CC_sub(1))

[-3.1918626098796556, 3.7072342233913163]
[1.5959313049398278, -1.8536171116956581]
-3.1918626098796556
3.7072342233913163
1.5959313049398278
-1.8536171116956581


Directional subgradients can be propagated too. In the case that subgradients are to computed along the direction (1,-1) for both the convex and concave relaxations, we define:

In [9]:
X.set_subgradient(1, 1,1)
Y.set_subgradient(1, -1,-1)
Z = X*(pymc.exp(X)-Y)**2
print("f relaxations and subgradients along direction (1,-1) at (-1,0):",Z)

f relaxations and subgradients along direction (1,-1) at (-1,0): [ -2.76512e+01 :  1.38256e+01 ] [ -1.75603e+01 :  8.78014e+00 ] [ (-6.89910e+00) : ( 3.44955e+00) ]


The FADBAD++ remains to be done here. 